# Car Residual Value Modeling
_**Part1: Analyze and summarize the data!**_

---

---

This is a problem of finding correlations in multiple time series (TS). Before we can find any correlations, we need to decompose the within-series dependences, without which spurious correlations will happen.

There are two within-series dependences: 1. Trend. 2. Seasonality, and we are going to remove both.

## Load libraries

In [392]:
import pandas as pd
from pandas_datareader import data
import datetime
import numpy as np
from bokeh.plotting import figure, show
from bokeh.palettes import viridis, Spectral6
from bokeh.io import output_notebook
import datetime
from pandas import Series
from sklearn.preprocessing import StandardScaler
output_notebook()

Loading BokehJS ...

## Data preprocessing (imputation, scaling, and change to TS type)

In [393]:
# Load data
data = pd.read_csv('/Users/meliu/Downloads/sample_data.csv')

# Define time series parameters
rng = pd.date_range('10/1/2005', periods=111, freq='M')

# Taking care of missing data
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
imputer = imputer.fit(data.iloc[:, :-2])
data.iloc[:, :-2] = imputer.transform(data.iloc[:, :-2])

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_data = StandardScaler()
data.iloc[:, :-1] = sc_data.fit_transform(data.iloc[:, :-1])

# Tranfer data to time series
data['actual_date'] = pd.to_datetime(data['actual_date'])
data.index = data['actual_date']
del data['actual_date']

# Data preview
data.head()

,Population_Civilian,CPI,CCI,Auto_Loans,Exist_HomeSales,Vehicle_Loans,Gas,GDP_Real,Housing_Starts,Indus_Prod,...,UMich_CCI,Unemp_Rate,Loan_Loss,cnt_M_U,cnt_M_N,cnt_S_U,cnt_S_N,cnt_mkt_U,cnt_mkt_N,BB_avg_value
actual_date,,,,,,,,,,,,,,,,,,,,,
2006-10-01,-1.683866,-2.012447,0.362871,5.946270e-16,2.895876,-0.723017,-1.365930,-0.866607,2.114364,0.347377,...,1.564016,-1.517373,-1.083306,-1.505885,0.656598,-1.573073,-0.630947,-1.582178,0.254309,2.246052
2006-11-01,-1.647896,-2.003114,0.371481,5.946270e-16,2.878400,-0.702292,-1.371177,-0.861386,2.390928,0.326357,...,1.424086,-1.462439,-1.079512,-1.505885,0.646455,-1.573073,-0.644597,-1.582178,0.245453,2.034131
2006-12-01,-1.616148,-1.900449,0.375782,5.946270e-16,3.018211,-0.681567,-1.243498,-0.856164,2.667492,0.508790,...,1.386772,-1.517373,-1.075719,-1.505885,0.644363,-1.573073,-0.622229,-1.582178,0.242176,2.034131
2007-01-01,-1.531383,-1.868996,0.376974,5.946270e-16,1.829820,-0.660842,-1.406157,-0.850942,1.827297,0.421868,...,1.871860,-1.407505,-1.071925,-1.684806,0.646455,-1.772533,-0.030191,-1.825951,0.555588,1.769230
2007-02-01,-1.502607,-1.795357,0.368660,5.946270e-16,1.917201,-0.447584,-1.329200,-0.786274,2.075855,0.623852,...,1.349457,-1.462439,-1.071925,-1.691591,0.644685,-1.758338,-0.436062,-1.834885,0.263231,1.663269


## Raw data plot

In [394]:
mypalette=Spectral6[0:numlines]

p = figure(width=1000, height=600, x_axis_type="datetime") 
color_ix = 0
for symbol in ['BB_avg_value','cnt_mkt_U']:
    p.line(data.index.values, data[symbol].values, legend=symbol, line_color=mypalette[color_ix], line_width=2)
    color_ix = color_ix + 1
show(p)

## Removing Trend and Seasonality

In [395]:
import statsmodels.api as sm

data.interpolate(inplace=True)
for symbol in ['Population_Civilian','CPI','CCI','Auto_Loans','Exist_HomeSales','Vehicle_Loans','Gas','GDP_Real','Housing_Starts','Indus_Prod','Manufac_Index','Bk_LoanRt','Prod_Index','Consump_Exp','Prod_Price_Index','HH_Debt','New_Auto_Loans','UMich_CCI','Unemp_Rate','Loan_Loss','cnt_M_U','cnt_M_N','cnt_S_U','cnt_S_N','cnt_mkt_U','cnt_mkt_N','BB_avg_value']:
    res = sm.tsa.seasonal_decompose(data[symbol])
    data[symbol] = res.resid

## After detrending and deseasonalizing plot

In [397]:
numlines=2
mypalette=Spectral6[0:numlines]

p = figure(width=1000, height=600, x_axis_type="datetime") 
color_ix = 0
for symbol in ['BB_avg_value','cnt_mkt_U']:
    p.line(data.index.values, data[symbol].values, legend=symbol, line_color=mypalette[color_ix], line_width=2)
    color_ix = color_ix + 1
show(p)

## Calculate Correlations

In [398]:
corr_df = data.corr(method='pearson')
corr_df

,Population_Civilian,CPI,CCI,Auto_Loans,Exist_HomeSales,Vehicle_Loans,Gas,GDP_Real,Housing_Starts,Indus_Prod,...,UMich_CCI,Unemp_Rate,Loan_Loss,cnt_M_U,cnt_M_N,cnt_S_U,cnt_S_N,cnt_mkt_U,cnt_mkt_N,BB_avg_value
Population_Civilian,1.000000,0.000876,0.031991,0.134878,0.063506,-0.186653,0.034484,0.127374,-0.075805,0.108672,...,0.272321,-0.102271,0.025480,-0.197417,0.456550,-0.360287,0.340167,-0.399012,0.245199,0.323208
CPI,0.000876,1.000000,-0.124502,0.069898,0.284922,0.333877,0.961084,0.339447,0.427415,0.303119,...,0.157883,-0.221303,-0.001930,-0.178060,0.192780,-0.150629,0.353334,-0.254057,0.339735,0.373300
CCI,0.031991,-0.124502,1.000000,-0.171519,-0.071247,-0.191290,-0.103781,-0.210777,-0.299758,0.148358,...,0.113057,-0.054455,-0.463461,-0.109552,-0.021388,-0.258944,0.000221,-0.021946,-0.117916,0.107037
Auto_Loans,0.134878,0.069898,-0.171519,1.000000,0.178282,0.022809,0.113070,0.168584,-0.046887,-0.018369,...,-0.032169,0.092989,0.099932,0.383835,-0.145415,0.282464,0.085711,0.161624,-0.036095,0.235075
Exist_HomeSales,0.063506,0.284922,-0.071247,0.178282,1.000000,0.168679,0.286188,0.131187,0.107490,0.068143,...,0.170152,0.006731,0.049496,-0.099719,0.156782,-0.047702,0.291819,-0.199933,0.304539,0.053967
Vehicle_Loans,-0.186653,0.333877,-0.191290,0.022809,0.168679,1.000000,0.389674,0.353592,0.248850,0.071999,...,0.082636,0.020459,0.237018,-0.203426,0.059075,0.064907,0.159081,-0.133832,0.240226,-0.001822
Gas,0.034484,0.961084,-0.103781,0.113070,0.286188,0.389674,1.000000,0.349623,0.436808,0.284725,...,0.182230,-0.224112,-0.034119,-0.139304,0.193142,-0.107614,0.371331,-0.212415,0.374824,0.307086
GDP_Real,0.127374,0.339447,-0.210777,0.168584,0.131187,0.353592,0.349623,1.000000,0.336799,0.522452,...,-0.077083,-0.214945,0.324516,-0.140621,0.346410,-0.093167,0.415056,-0.221476,0.406967,0.019178
Housing_Starts,-0.075805,0.427415,-0.299758,-0.046887,0.107490,0.248850,0.436808,0.336799,1.000000,0.194759,...,-0.012466,-0.153310,0.102945,-0.062140,0.103825,0.007997,0.182042,-0.052201,0.220107,0.050673
Indus_Prod,0.108672,0.303119,0.148358,-0.018369,0.068143,0.071999,0.284725,0.522452,0.194759,1.000000,...,-0.146439,-0.475179,-0.212624,-0.187941,0.330495,-0.302607,0.381794,-0.253984,0.319716,0.211051


## Create a Heat Map

In [399]:
corr_matrix = corr_df.as_matrix()
N = len(symbols)
factors = list(symbols)
x = []
y = []
colors = []
for i in range(N):
    for j in range(N):
        x.append(symbols[j])
        y.append(symbols[i])
        cor = corr_matrix[i, j]
        rgb = (int(abs(cor) * 255), 0, int((1 - abs(cor)) * 255))
        colors.append('#%02x%02x%02x' % rgb)

p2 = figure(x_range=factors, y_range=factors)

p2.rect(x, y, color=colors, width=1, height=1)

show(p2)

## Close Inspection of the BB avg value column

In [400]:
corr_df['BB_avg_value']

Population_Civilian    0.323208
CPI                    0.373300
CCI                    0.107037
Auto_Loans             0.235075
Exist_HomeSales        0.053967
Vehicle_Loans         -0.001822
Gas                    0.307086
GDP_Real               0.019178
Housing_Starts         0.050673
Indus_Prod             0.211051
Manufac_Index          0.282144
Bk_LoanRt              0.405824
Prod_Index             0.213602
Consump_Exp           -0.114690
Prod_Price_Index       0.280012
HH_Debt                0.023147
New_Auto_Loans         0.097343
UMich_CCI              0.086643
Unemp_Rate             0.000985
Loan_Loss             -0.041441
cnt_M_U               -0.129811
cnt_M_N                0.122820
cnt_S_U               -0.288987
cnt_S_N                0.249418
cnt_mkt_U             -0.290480
cnt_mkt_N              0.068545
BB_avg_value           1.000000
Name: BB_avg_value, dtype: float64

# deep color areas are of interests (Good Economy 101 for me):

1. The car count red square in the heat map show strong correlation between the 6 numbers. This is easy to understand, since model, segment, and market, old or new, are all strongly related.
2. Manufacture index and industry production: both shows the strength of the economy.
3. Loan loss and HH debt: When household debt is high, loan loss rate is high in order to control risk.
4. Consump_Exp and Prod_Price_Index: When consump_exp is high, PPI is high.
5. Gas and CPI: Gas price is a good indicator for inflation.
6. Prod_Price_Index and CPI: These are also two close related metrics, which measures prices.

# What influence car residual price?
Top 3: Population, CPI, Loan Rate , and new car counts lead to price increase, and used car counts lead to price decrease.